# Import Libraries

In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 15.4 MB/s eta 0:00:00


In [8]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
import cv2
from PIL import Image
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
import easyocr

In [10]:
# Step 1: Prepare the Dataset
train_csv_file = 'DataTrain.csv'
test_csv_file = 'DataTest.csv'
train_image_folder = os.path.join(os.getcwd(), 'DataTrain')
test_image_folder = os.path.join(os.getcwd(), 'DataTest')

# Step 2: Load and Preprocess the Dataset
current_directory = os.getcwd()
train_csv_path = os.path.join(current_directory, train_csv_file)
test_csv_path = os.path.join(current_directory, test_csv_file)
train_image_folder_path = os.path.join(current_directory, train_image_folder)
test_image_folder_path = os.path.join(current_directory, test_image_folder)

df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

train_image_paths = df_train['NameofFile'].apply(lambda x: os.path.join(train_image_folder_path, x)).values
train_labels = df_train['Vehicleregistrationplate'].values

test_image_paths = df_test['Name of File'].apply(lambda x: os.path.join(test_image_folder_path, x)).values

# Label encoding
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)

# Preprocess the images (e.g., resize, normalize, convert to tensors)
image_size = (224, 224)

train_images = []
for image_path in train_image_paths:
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    train_images.append(image)
train_images = np.array(train_images)

# Calculate the number of plate classes
num_classes = len(label_encoder.classes_)

# Step 3: Perform Character Segmentation
def perform_character_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Convert image to 8-bit unsigned integer
    binary = np.uint8(binary)

    # Apply connected component analysis
    connectivity = 8
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary, connectivity, cv2.CV_32S)

    # Extract individual character images
    segmented_characters = []
    for i in range(1, num_labels):
        x, y, w, h, area = stats[i]
        if area > 100:  # Filter out small components
            character = image[y:y+h, x:x+w]
            segmented_characters.append(character)

    return segmented_characters



# Step 4: Build the Model Architecture
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

batch_size = 32
epochs = 15

train_characters = []
train_labels_extended = []
for i in range(len(train_images)):
    image = train_images[i]
    characters = perform_character_segmentation(image)
    train_characters.extend(characters)
    label = train_labels[i]
    train_labels_extended.extend([label] * len(characters))

train_characters = np.array(train_characters)
train_labels_extended = np.array(train_labels_extended)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_characters, train_labels_extended, batch_size=batch_size, epochs=epochs)

# Step 6: Evaluate and Fine-Tune

# Step 7: Test the Model
# Make predictions on test images
test_characters = []
for image_path in test_image_paths:
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    characters = perform_character_segmentation(image)
    test_characters.extend(characters)

test_characters = np.array(test_characters)
predictions = model.predict(test_characters)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Print predicted labels
for image_path, predicted_label in zip(test_image_paths, predicted_labels):
    print(f"Image: {image_path} - Predicted Label: {predicted_label}")

Epoch 1/15
25/25 [==============================] - 737s 28s/step - loss: 6.5216 - accuracy: 0.0100
Epoch 2/15
25/25 [==============================] - 701s 28s/step - loss: 6.2492 - accuracy: 0.0200
Epoch 3/15
25/25 [==============================] - 700s 28s/step - loss: 6.0475 - accuracy: 0.0162
Epoch 4/15
25/25 [==============================] - 663s 26s/step - loss: 5.7888 - accuracy: 0.0275
Epoch 5/15
25/25 [==============================] - 650s 26s/step - loss: 5.3771 - accuracy: 0.0338
Epoch 6/15
25/25 [==============================] - 644s 26s/step - loss: 4.9201 - accuracy: 0.0600
Epoch 7/15
25/25 [==============================] - 639s 26s/step - loss: 4.5468 - accuracy: 0.0887
Epoch 8/15
25/25 [==============================] - 646s 26s/step - loss: 4.1109 - accuracy: 0.1163
Epoch 9/15
25/25 [==============================] - 646s 26s/step - loss: 3.6126 - accuracy: 0.1800
Epoch 10/15
25/25 [==============================] - 630s 25s/step - loss: 3.2473 - accuracy: 0.2325